<a href="https://colab.research.google.com/github/mimilazarova/DD2424-covid-xray-project/blob/master/covid.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
import matplotlib.pyplot as plt
import cv2
import pickle
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import train_test_split
import os

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
with open('/content/drive/My Drive/DD2424_Project/data/testX.pickle', 'rb') as f:
  testX = pickle.load(f)
with open('/content/drive/My Drive/DD2424_Project/data/test_y.pickle', 'rb') as f:
  test_y = pickle.load(f)

testN = testX.shape[0]
testX = testX.reshape((testN, 224, 224, 1))
testX.shape

(231, 224, 224, 1)

In [9]:
dirname = "/content/drive/My Drive/DD2424_Project/split-80-20-bs16-with-augmentation-weights-1-1-8"
filename = "model8-3-augmentation-16-0.78.hdf5"
m = keras.models.load_model(os.path.join(dirname, filename))

test_y_pred_m = m.predict(testX)
test_y_pred8 = np.argmax(test_y_pred_m, axis=1)

conf_matrix = confusion_matrix(test_y, test_y_pred8, labels=[0, 1, 2])
print("Test set")
print(conf_matrix)
print(classification_report(test_y, test_y_pred8, digits=3))

OSError: ignored

In [7]:
filename = "/content/drive/My Drive/DD2424_Project/model8-16-0.91.hdf5"
loaded = keras.models.load_model(filename)
test_y_pred_m = loaded.predict(testX)
test_y_pred8 = np.argmax(test_y_pred_m, axis=1)

conf_matrix = confusion_matrix(test_y, test_y_pred8, labels=[0, 1, 2])
print("Confusion matrix:")
print(conf_matrix)
print(classification_report(test_y, test_y_pred8, digits=3))

Confusion matrix:
[[95  5  0]
 [11 89  0]
 [10 13  8]]
              precision    recall  f1-score   support

           0      0.819     0.950     0.880       100
           1      0.832     0.890     0.860       100
           2      1.000     0.258     0.410        31

    accuracy                          0.831       231
   macro avg      0.884     0.699     0.717       231
weighted avg      0.849     0.831     0.808       231

